# Example processing script for SLSTR data
### A simple example for retrieving sea surface reflectance using data from the Sentinel-3 / SLSTR instrument.

This script requires SLSTR data, which is available for download at https://scihub.copernicus.eu/

In this example we compute the sea surface reflectance based on assumed constant winds across the swath. We do not apply and cloud or land masking and we do not calculate any of the BRDF parameters.

In [ ]:
%load_ext autoreload
%autoreload 2

from satpy import Scene, find_files_and_readers
from pycoxmunk import PyCoxMunk
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set which bands we wish to load
bnames = ['S1', 'S2', 'S3']

# Set the directory containing the SLSTR data dir
# This folder should contain one or more subfolders with the SLSTR data (ending .SEN3)
# To save memory and CPU time, only use a few granules maximum for this.
indir = 'D:/sat_data/slstr/'

# Set the directory where we'll save the output
outdir = 'D:/sat_data/slstr/out/'

In [ ]:
# Create a scene by searching for files
scn = Scene(find_files_and_readers(base_dir=indir, reader='slstr_l1b'))
# Load the bands
scn.load(bnames)

# Additionally load the solar and satellite angles
# This saves computing them internally
scn.load(['solar_zenith_angle',
          'solar_azimuth_angle',
          'satellite_zenith_angle',
          'satellite_azimuth_angle'])

In [ ]:
# Create the PyCoxMunk class for processing
pcm = PyCoxMunk(scn, bnames)

In [ ]:
# Here we assume a uniform wind, needed for cases we don't have actual wind data
# See SEVIRI example for case where wind data is available
pcm.setup_wind(1., 1.)

In [ ]:
# Retrieve the sea surface reflectance
pcm.retr_coxmunk_refl()

In [ ]:
# Save results to the directory specified earlier
# Each dataset is saved to its own file.
pcm.scn.save_datasets(base_dir=outdir, enhance=False, dtype=np.float32)

In [1]:
from pycoxmunk.CM_Shared_Wind import CMSharedWind